In [ ]:
from scipy.io.wavfile import read as read_wav
from IPython.display import Audio
import matplotlib.pyplot as plt
from collections import Counter
import tensorflow as tf
import librosa as lr
import seaborn as sn
import pandas as pd
import numpy as np
import scipy
import pydub

In [ ]:
def predict(features):
    
    result = []
    
    lstm_1_res = np.argmax(lstm_1_model.predict(features))
    # print('LSTM-1:', lstm_1_res)
    
    result += [lstm_1_res]
    
    # strong classes:
    if lstm_1_res == 0:
        
        lstm_2a_res = np.argmax(lstm_2a_model.predict(features))
        # print('LSTM-2a:', lstm_2a_res)
        
        result += [lstm_2a_res]
        
        # substrong-1:
        if lstm_2a_res == 0:
            
            lstm_3a_res = np.argmax(lstm_3a_model.predict(features))
            # print('LSTM-3a:', lstm_3a_res)
            
            result += [lstm_3a_res]
            
        # substrong-2:
        else:
            
            lstm_3b_res = np.argmax(lstm_3b_model.predict(features))
            # print('LSTM-3b:', lstm_3b_res)
            
            result += [lstm_3b_res]
            
    # mild classes:
    else:
        
        lstm_2b_res = np.argmax(lstm_2b_model.predict(features))
        # print('LSTM-2b:', lstm_2b_res)
        
        result += [lstm_2b_res]
        
        
        # submild-1:
        if lstm_2b_res == 0:
            
            lstm_3c_res = np.argmax(lstm_3c_model.predict(features))
            # print('LSTM-3c:', lstm_3c_res)
            
            result += [lstm_3c_res]
            
        # submild-2:
        else:
            
            lstm_3d_res = np.argmax(lstm_3d_model.predict(features))
            # print('LSTM-3d:', lstm_3d_res)
            
            result += [lstm_3d_res]
            
    return lookup(list(result))

# Accuracy on GTZAN Validation and Test Set:


In [ ]:
processed = np.load('/content/drive/MyDrive/audio-genre-classification-main/dataset/prepared/processed.npy', allow_pickle=True)


In [ ]:
## Top-1 Accuracy:


In [ ]:
top_1_confusion_mat = np.zeros(shape=(10, 10))


In [ ]:

ground_truth_labels = []

for i in range(0, 5000, 5):
    
    if (i % 500) >= 5 * 90:
        predicted = []
        for j in range(0, 5):
            if ((i+j) % 500) >= 5 * 90:
                predicted += [predict(processed[i+j]['features'].reshape(1, 256, 40))]
        
        original = processed[i]['ground_truth']
        
        top_1 = Counter(predicted).most_common(n=1)
        
        if original == top_1[0][0]:
            top_1_confusion_mat[genres[original]][genres[original]] += 1
        else:
            top_1_confusion_mat[genres[original]][genres[top_1[0][0]]] += 1
        
        ground_truth_labels += [original]
        
        print('=========================================')
        print('Original: {}'.format(original))
        print('Predicted: {}'.format(top_1))


In [ ]:

top_1_frame = pd.DataFrame(
    data=top_1_confusion_mat,
    index=list(genres.keys()),
    columns=list(genres.keys())
)


In [ ]:

fig = plt.figure(figsize=(8, 6))

sn.set(font_scale=1.4) # for label size
sn.heatmap(top_1_frame, annot=True, annot_kws={"size": 10})

fig.tight_layout()
plt.show()


In [ ]:

top_1_accuracy = 100 * (np.sum(np.diag(top_1_confusion_mat)) / np.sum(top_1_confusion_mat))
print('Top-1 Accuracy: {} %'.format(top_1_accuracy))

In [ ]:
## Top-2 Accuracy:


In [ ]:


top_2_confusion_mat = np.zeros(shape=(10, 10))


In [ ]:


for i in range(0, 5000, 5):
    
    if (i % 500) >= 5 * 90:
        predicted = []
        for j in range(0, 5):
            if ((i+j) % 500) >= 5 * 90:
                predicted += [predict(processed[i+j]['features'].reshape(1, 256, 40))]
        
        original = processed[i]['ground_truth']
        
        top_2 = Counter(predicted).most_common(n=2)
        
        if original in [top_2[i][0] for i in range(len(top_2))]:
            top_2_confusion_mat[genres[original]][genres[original]] += 1
        else:
            for i in range(len(top_2)):
                top_2_confusion_mat[genres[original]][genres[top_2[i][0]]] += (1 / len(top_2))
        
        # print('=========================================')
        # print('Original: {}'.format(original))
        # print('Predicted: {}'.format(top_2))


In [ ]:

top_2_frame = pd.DataFrame(
    data=top_2_confusion_mat,
    index=list(genres.keys()),
    columns=list(genres.keys())
)

In [ ]:
fig = plt.figure(figsize=(8, 6))

sn.set(font_scale=1.4) # for label size
sn.heatmap(top_2_frame, annot=True, annot_kws={"size": 10})

fig.tight_layout()
plt.show()



In [ ]:
top_2_accuracy = 100 * (np.sum(np.diag(top_2_confusion_mat)) / np.sum(top_2_confusion_mat))
print('Top-2 Accuracy: {} %'.format(top_2_accuracy))

